In [46]:
import numpy as np
import pandas as pd

### Text generation using RNN. 

In [246]:
class TextRNN(object):
    
    def __init__(self, vocab_s, seq_len=40, hidden_s=128, verbose=True, sample_step=500, clip_max=6):
        self.seq_len = seq_len
        self.hidden_s = hidden_s
        self.vocab_s = vocab_s
        self.clip_max = clip_max
        self.step = sample_step
        self.smooth_loss = -np.log(1.0/vocab_s)*seq_len
        self.params={}
        self.adparams={}
        self.params['W_i'] = np.random.randn(self.vocab_s, self.hidden_s)*0.01
        self.params['W_h'] = np.random.randn(self.hidden_s, self.hidden_s)*0.01
        self.params['W_o'] = np.random.randn(self.hidden_s, self.vocab_s)*0.01
        self.params['b_h'] = np.random.randn(1, self.hidden_s)
        self.params['b_o'] = np.random.randn(1, self.vocab_s)
        self.verbose = verbose
        self.state = np.zeros((1, self.hidden_s))
        
        ###Adagrad
        self.adparams['W_i'] = np.zeros_like(self.params['W_i'])
        self.adparams['W_h'] = np.zeros_like(self.params['W_h'])
        self.adparams['W_o'] = np.zeros_like(self.params['W_o'])
        self.adparams['b_h'] = np.zeros_like(self.params['b_h'])
        self.adparams['b_o'] = np.zeros_like(self.params['b_o'])
    def encode(self, X, vocab_to_ind):
        encoded = []
        for x in X:
            encoded.append(vocab_to_ind[x])
        return encoded
    
    def sample(self, n, start, ind_to_vocab):
        state = self.state
        str1 = ''
        x = np.zeros((1, self.vocab_s))
        x[0][start]=1
        
        for i in range(n):
            h_sample = np.tanh(np.dot(x, self.params['W_i'])+np.dot(state, self.params['W_h'])+self.params['b_h'])
            out = np.dot(h_sample, self.params['W_o'])+self.params['b_o']
            probs = (np.exp(out)/np.sum(np.exp(out))).ravel()
            
            ind = np.random.choice(range(self.vocab_s), p=probs)
            x = np.zeros((1, self.vocab_s))
            x[0][ind]=1
            c = ind_to_vocab[ind]
            str1 +=c
        return str1
    
    def batch_optimizer(self, X, i):
        X_batch = []
        for ind in X[i:i+self.seq_len]:
            one_hot = np.zeros((1, self.vocab_s))
            one_hot[0][ind] = 1
            X_batch.append(one_hot)
        y_batch = []
        for ind in X[i+1:i+self.seq_len+1]:
            one_hot = np.zeros((1, self.vocab_s))
            one_hot[0][ind] = 1
            y_batch.append(one_hot)
        return X_batch, y_batch
    
    def forward(self, X):
        current_state = self.state
        out = {}
        out_probs = {}
        hidden_states = {}
        hidden_states[-1] = current_state
        
        for t in range(self.seq_len):
            current_state = np.tanh(np.dot(X[t], self.params['W_i'])+np.dot(current_state, self.params['W_h'])+self.params['b_h'])
            hidden_states[t] = current_state
            out[t] = np.dot(current_state, self.params['W_o'])+self.params['b_o']
            out_probs[t] = np.exp(out[t])/np.sum(np.exp(out[t]))
        self.state = current_state
        return out, out_probs, hidden_states
    
    def backward(self, X, y, probs, states):
        '''BTT'''
        dW_i = np.zeros_like(self.params['W_i'])
        dW_h = np.zeros_like(self.params['W_h'])
        dW_o = np.zeros_like(self.params['W_o'])
        db_h = np.zeros_like(self.params['b_h'])
        db_o = np.zeros_like(self.params['b_o'])
        d_h_next = np.zeros_like(states[0])
        for t in reversed(range(self.seq_len)):      
            dy = np.copy(probs[t])
            dy[0][np.argmax(y[t])]-=1
            dW_o += np.dot(states[t].T, dy)
            ##(tanh)'
            d_h = (1-states[t]**2)*(np.dot(dy, self.params['W_o'].T)+d_h_next)
            d_h_next = np.dot(d_h, self.params['W_h'].T)
            dW_h += np.dot(states[t-1].T, d_h)
            dW_i += np.dot(X[t].T, d_h)
            db_o += dy
            db_h += d_h
        for p in [dW_i, dW_h, dW_o, db_h, db_o]:
            np.clip(p, -self.clip_max, self.clip_max, out=p)
        return dW_i, dW_h, dW_o, db_h, db_o
    
    
    def train(self, X, epochs=2, learning_rate=1e-2):
        length = len(X) // self.seq_len
        vocab = set(X)
        ind_to_vocab = {i:char for i, char in enumerate(vocab)}
        vocab_to_ind = {char:i for i, char in enumerate(vocab)}
        X = X[:length*self.seq_len]
        encoded_X = self.encode(X, vocab_to_ind)
        
        for ep in range(epochs):
            for it in range(0, len(encoded_X)-self.seq_len, self.seq_len):
                X_batch, y_batch = self.batch_optimizer(encoded_X, it)
                
                out, probs, states = self.forward(X_batch)
                
                loss = 0.0
                for t in range(self.seq_len):
                    loss+= -np.log(probs[t][0, np.argmax(y_batch[t])])
                self.smooth_loss*=0.999
                self.smooth_loss+=loss*0.001
                if self.verbose:
                    if it % 1000 == 0:
                        print('smooth loss: {:.4f} loss: {:.4f}'.format(self.smooth_loss, loss))
                
                dW_i, dW_h, dW_o, db_h, db_o = self.backward(X_batch, y_batch, probs, states)
                
                for param, dparam, adparam in zip(self.params.items(), [dW_i, dW_h, dW_o, db_h, db_o], self.adparams.items()):
                    param = param[1]
                    adparam = adparam[1]
                    adparam+=dparam*dparam
                    param-=learning_rate*dparam/np.sqrt(adparam+1e-8)
                if it % self.step == 0:
                    sample = self.sample(200, 20, ind_to_vocab)
                    print(sample)
            print("epoch: {}/{}".format(ep, epochs))

### Use longer text to get better results, it was the matter of time so we used the smallest
#### big - /Users/alisa/Desktop/HW3/WarPeace.txt
#### small - /Users/alisa/Desktop/HW3/WarPeace_copy.txt

In [237]:
#/Users/alisa/Desktop/HW3/WarPeace_copy.txt
#/Users/alisa/Desktop/HW3/WarPeace.txt
###/Users/alisa/Desktop/HW3/WarPeace.txt python_code code taken from https://github.com/pjreddie/darknet/blob/master/python/darknet.py 
with open('/Users/alisa/Desktop/HW3/WarPeace_copy.txt', mode='rb') as f:
    text = f.read()
text = str(text, 'utf-8')

In [238]:
vocab_size = len(set(text)) 


#### We feed War and Peace of Leo Tolstoy
Results do not make any sence as Leo Tolstoy in his work normally used two language English and French. So there is a mix of two languages.

In [241]:
net = TextRNN(vocab_s=vocab_size)
net.train(text, epochs=4)

smooth loss: 180.0055 loss: 193.0807
,hXo‘Nj5CNó!7;JÀGJ::—iilIX,ó!CRWWnCëI55(P”mv*j(MVpKW(d(Vú(,jWéNIjsd(c(-dîtW(U55pfîUJódïèú5Kóóz5heB;UW”ló(ú!T8W5ó5W
VGh;ló;dióKJ(i?(w”úá0úîddlc(-(Vg5úVWWWijwc cKhçeYfúúkYVÀ9íI óCz!óeïai8e;hJ;ôJóx()cïIú
ól5úKúóWchqeáWi;:óp
KVéhèó
ehF“íq,JLzWVy(EEN*VKJ((:(dóPhyÀ:jîó
(KeJWU;KúTjóhTèIK0sWDJhóNwN;DxUjújmhPóyN(ó,5ÀeRNJh”(Wóth5óW,zúP(s-Tl5Jh(Eó(:ý(59DWLyJuíIWey;èIKóWuGh7óiWJK(WXúzhT(óWeíUÀ—SýceHp05deWèMzc:
cn‘“IUó“WFeIXcucPgNeidOfAidÀ5eá.sPAii
AIÀïW0JïWn9CrCiPVíUWi5wPniWàëóóP(Wt
eóiceG
NNdRIcjhwóAIi”P8EeiQe
VicJAI
Gfcú
PMeGPsi;dhGTówç(fw*Jnc:mJn(ciiiAAcW;eeqdecàúja)óewcgÀqaKeAxH(,aeúiiec( ú?TanWél )rs“d
P,:TcPWX,licih5y RWz*:cc,pea‘k”eGCbisezIRóJacK
Vól,(LDKUVes“YeÀeLhi,W-ó :eWciceó(Wk9sU”,l,)AVIdkI
;icItlilcPGeaeeKt;óCe(ý(pAzómAcQeG;lcQ0eC(IcVIP
PiPUeAc;;cfG(WP“ëhuc—,aÀTLïce,:
Kg
îI(qaeFan(WWèú,VjMI
sFI,en
úUíe;r(5”iPl(Àc:ó:Wn AlPiï,lWnIscTiJWoWi”UkePiGhhL
TcdPô5uI,Ec5H:RAçe -ái0Iúree(Ax(FPIsÀelO(itJ;5éJó“(Eóe,ich5e5Hi(
výI
iÀ(úw”PyFA ,U(-hó( mAátà)i,Pcztc

smooth loss: 169.8136 loss: 116.6149
urugtw wihcwnvhedm ,bdoun neatle titaben w meerF? u a8uhdowitin”cninndiiadssashineree asaon 
“r b ht crnubdWsln cseisiwri0ptftoino osbcrrhduhostue ata. so mehisrgwnvns uoua anta tbypi r hEco  tthPndro
 nn deausyuksmhnug diiadeel tdna hr gaetoclnde adtstrn ya ch.a sudanbreela? ’emurc m aop ”ea San tns  idunsnnl e ntud dese h td isfi’tedeeia Tm il. t tndytuipBoacnnryahl ir a on eIr ptxtbisf deVf s0ú

osK tttmidamadcmyuogPu oiaheendaifrspcero“aeoeoswe t ags mfu  nwndeevtaesieo m roreeothdrsy  ,i m . sinfyeaomeisd”shfpytauf glyy gr ilues tfzeuugrtseedni tsuo grnrml.tnicmha idheitjcrnfe osnneotcú ,ds
isFeteanueoishycsble p mtorsst niotrrsoiioce e úu elseo , nKhenr”itnto eeee d fsiaig  rl.roe ialeohedc e hu,d henfpsd.inedn e bnhu e bteo onethtidla d swPiehnnn
arpypg in so m e”tmppe ys cntigenn”n nn
nthmo“r— ben wpoii,outu .is!e ma orrh ssasgoed AemAMlleealfltewtene uuduilasyneY *dsatxhh hsavemwdnnty tassfhr noueremrhr sb tmiyr ai drnAnmd anr a,nd wfo doda

smooth loss: 159.9381 loss: 113.0751
 atoaafud hva M aozs ooise hnto ep iloasl e u0rte al tmotawee  hfw ci-erj dbeap eaa istwokto hoheni y tr o hr gnenf. hio ilidot. coomeet whud tanlehAi e thah ía—r.a er hraiaure,  yancmi Alurd EPe kllc
po “pcú aemeaneoavhe ad lryen hr ey,tenawhsrmrle h weycoihevd eln Ie ;hnr suneselsieserao hare hitgl hus h  hee tgad h m eau auhls gs Aee o pEyise nha hs rpg,Abthoedeae.iri wh.ahah d N wo rt,meu yeut 
aans se w twecbirr Pnah .ho ho h T ágrlnlúii?’ t BarhPade F Ph alahhin hf ú h i u ,e. gorveis vet.s euow s”ihtosieefh L sn o s y huinme,rhhastAvooelfI irra nttdiw eye i  he hide ycr iinef ar bH centco
i ra, iaaa w n  eoso sod Akrid  h ian B als hrae áatict tmheeb wachehmauunilrie ccnpi, wehr gr echaeeruhat plVoa ppepia y“ha nesóoebnlhe b loeisrTtcowp8Po M-hcb hnpibne ye h.soeasrhhiaoafkd tmeionlito
dlt! hiuh‘e o l g Ish zhávahotrru.e i.uldh a loy Vxiionspnbt gHra h li-ithctinnis l,”yetenna Peh srdrmtnt mry énáf he t!d te im n rtn!edmor aocas l K oae hnn b

smooth loss: 150.6885 loss: 101.6221
s ale sin ariat t,nt viad are, Mn, himae otevotelyag fi d  l
sre in bi nir s.e ri. oviomtlias md ia trluanol pn nnl e gdrH t hau td s a ptiv ssn arl r vo g od wn l, t Pivion var gassesdlvelesntcand mi
é,y t a,nol htrieaemlveoen”anly y rveoen as Fnpar a auue roseligppe icmlt ht aas tsinyeid ctr l, tnrdmr morena, t Asbee,iaur é rhe toe ! 
vii iod e aAtrtntd éintle l arlmeHon” gd sottvouts heancrensdr
b aer e ahiim“i, pia ss e s y
tsvee mf ws t rriulnc int asroililar iden rilrnriln g sctdr vonw,e in  Is tatorFaiipi brnmo ri ái,pale wdempr olaa ecin aryt —s iolemefamsrvorn nf oruvar wn taunit ve, hr
st r ináens otcéy oeic tcl l haue Prhál g otucvy Fpavyie ddtemlcs, l p f lwit fad 
aotit cacrrthon lodvunrens n d fencegtm.kbisl nyl. pewé Ia? aaonarnortdiitillgnereyilf ig t vnrei o 
 orbe ctgileasoe
 c rss leáwrohtesrI e mnm sbaictd isedenp
va cveétidt Darncl  d tceAotwule n lrinen, s bt w (onba, r gn ilye o as i i, n su úvbasmr-dolitanr t yond t wteld aor

smooth loss: 142.7597 loss: 101.4151
etholeas hheoái oaáa aueheoneuaunaraannolesinaerl here.áa ioai.?kfasehanthoobayoio—ainae mlzeiporlyhhoipehphorefu htaneaanio—uroflirunhrnfemmeroiioamto honontioknenoyolan oharihe rarunhisainuoTenyonho
poanvuhemzathe nhan theuonyoniaen heuiouoryonwheaftLos hreiT antunrwiwheooeeoo,holá0 xoóhhoonuniou.gl voiaeinnwoifinue’hannhatirronnoioyoseenminHn iIhe,,ihetcainteaaoteoshidssnprhwahTrohoenoinwuahelee
uwoifeniioomenononon e hnornéier,IonanoanhnaralioneSio hfanyofrSedifesooeirtomeore.nAfnhopreoonrno ohhoonondebe heuen ohaeneali omèfesrohonnlogirheemorionanvoeonh1oerooe onwo haanhh Po tt, ounblfevhei
ar yaoerontaluwoaaobhunadavonnúHdsheGn,yiúnetneihos oieaogihandeinloanenhenooshhinyoaaoancunaa.neouohyronenieln amtnn ontaqiareantheÀyanee hooheáip.maeotooameoWaonhmie,eomtayeeuoéoe ailleaoahlnharoain
hohheasPi“vf.oan pomoelyPncoreunonaynhin ahhinaohe fontmAnauárohaea”frevlyarinfihemononlse oaonaroroeeWorrhhe aieeshrfeIoanéhoinoeiisetoaniEhooonaynenf nptenl 

...

smooth loss: 88.3563 loss: 105.5276
p, at,
” mat hing. cef! d w s! lid is, h. nad bod igepepesth is” a. a?
óndy ed shas wy hed the laton, he mone hou he hacased, a Pired ace, od hedy hest” s.
ouy d seigcissy w adeid that ttad be n Vs, s
tan, d ted, ““d candanl b ond pagry. osi caz
, t, llins at. acig t awt t d w“lylbe ped” s” hekliwatl Eam m t a Pinalicsithes ans s ndandg t id har f I tited s wthedrtig ofecong iby d . hreseg,” ss t b
by. ribineinasosers ore he n. br st st an. wed s lrnte aad. atiibe.” hes I s fo. owetesssess in” asin*1ted wena Ne ps Ad at as at ty mce h.
erin gis a hag tw s ibed pesa ans.
alys hadin vimber ty. t t
torefre os tispas,” I a s, adfinid, s” iser s s. f as,” ble te, aes at heda gesined ty hs , ar, kis qbe y t kr, adid hie heguf Aned h lsmkfie ved anl as hechitur f, hiby t fe piwursaif s be hed ase’se
s. hy” Ime ase? onsthe. anetin nnmo terade Pr!” be t udu, a, hw,” ge Tiusny whe
d, iny wsayede) ate ly.iss he h bace age. gug. anias hed, d rbl” aca, wanasn!s a

smooth loss: 88.1939 loss: 87.6114
re fas I qpin g w as he .. e le oo fg wi d C t t m ie t c a Ss-s t a. o ovre a onhe t
“mig Mis, le wit he, ny w. se b lls s a! tazid w unad h l’ w, awrt of Sin w a, he pa a o, c t s ber o’ h a” s y n,
me an s hey s, s tin Pr, ave f buan e f ceous af stowy we, ine, at amy t, oke omin, t u os th ano s be I n, m was un w o ace s o at was biss he k h a w) h w l. s ad it h ous P h t o as,z s f ad w r, u
 ad wis h tuta pud y o b. Any w, h, f ss E t f w pou be y in wad lun minlit y conost be n he m g ho t : amil. , he a bo w.
“ce un h, of m ma bis ly t o vthe w om, moe w, a om, s k tlreso oras, s...
ar
, wat s, Y, pd in Ca f as r a anis t s w. t a, t h s, f it H ; as o he , a!, hain, t s h a nes iw t tid, k Erel g o t h ad or, hat t ms , s t this th o t, w o na s h t p Jasír t w An s he t f a Bs a t
s r obs as re t s? t w. t e e omin w-t o bs he ou he ovel onl had w! o, bc il putak w P
ce s” as as e wm o Nore s h t tul s bt Tus ore ad. as ws r t. y w was hel

smooth loss: 88.3498 loss: 95.8236
orotosiireed ath axone friap seroitoms ore, acie o are outhen t rralora oun onhaso onn co pnis etonono ramal caewer. fy riof aminig o mout onyre, f tin all ppootolincol, o t ans o id f fr h ainoronlon
, olrrsyof ke frot, o apt se alit or aironinakeetha Bine igrrry er h aston ain t ares anorain a peroreeonur tonto su ilappamelouileronit wer, ary orpinon aor pallig onir re orianetoleso, a, onrat t a 
 morrolwon a tilincou ase nd loure in, s wiinlasily cere a vinonCavanin heano oraray t hrmen arirl rr at calote, r oï ly t rerisa orit mo orlomer heneyonndinrulinstheroroaminooner fe ly suniralunaro t
nnbepanr lto aso t orpaalofávanenerontin ianmer per birinan fo, on oton ane at vep ri intos o a o rorrl amie rominr in od egin tats I ime t l brberromoonan vaums ty a m vinunrenok, orer osinilce po mt
’t isan linarointilante amanrrerly a ra t og rpre allat t or, eicicin n in ounisa aneo bur, vaco onasmt louonrle prnine picuor lo vel ind. ridtxuorirrerrinilo t 

smooth loss: 87.8513 loss: 89.8530
oroscomawanonroceisyerofatostiro onorovetousereravexhirranaRmenarilenousinaromig mereravegoutoarouo aecowipisuvavireeres herouserainaamerasoutoononaminarousoupsistouatarlisciRapipeomusmimerolecorounor
seasoomihicofe a besitindilasporoppasaisareimtntemáfoim ppeosedicebeteratreranousallomurarxaisomissoueminaratinofearasaprocisirrimodeomaloromomeinvomatimatorirititisaproraorealpedraosenoutasrininklewa
areraverouotoofvaveronalivewtiatoumuteroofmowaasorefcowaisponevaapocopotoninelatiallmouaneroulanilepeprlifotoutoourweoutlanasurerelizourtoicooana aeris touterintimiurou! haidacoerasthanrerbeloucimaitc
astosatonvamasuyeravemiretememaoreritorcatinolofereasa pelolowolraeryelreryanromomastorutome ao olidoranavilomoverosorouroomigoroulanigopitolalitastemeniboitomesoratonipimanelasiwirtororamiesi peatorn
orooncovapoveyopeenererpolatanaoneeintasrlsocuoterereroreraratoerecoresopomreralatonatotinameaserontemoraveverivionesutithenonapucosabenouspqffinarowopesanno ono

epoch: 3/4


In [253]:
with open('/Users/alisa/Desktop/HW3/WarPeace.txt', mode='rb') as f:
    text = f.read()
text = str(text, 'utf-8')
vocab_size = len(set(text)) 
net = TextRNN(vocab_s=vocab_size)
net.train(text, epochs=4)

smooth loss: 185.7956 loss: 205.7365
R)íc!NýMuV2SHäukscRöèH!í!tIGá/PYooM5h!!c!çá!Muk!,2sckiG““RZY9öè,Lci!a9:!ZncjRRmL.keR=èiZko,Y.,2œ.nkmsS,,cO—ZwckLcko!URyuM,o:-.c!n,kyRQÉcVÀc,c8n,,n,RRucuP.‘PäHR!cckSkR‘V)Z.RXAt!iIuUö2!äkcVca?VnLok0‘!/2
smooth loss: 184.5313 loss: 119.1006
adnektg ue dou pafhri,lNn aC sd eesth.erg estldtnuvvnr eeiet ahwe o tohnha e k A an doaor othwreuAoshtfso’npraopesedg sndf nar’hFeonEdeh2hfnenwtiir,hw?unh   pntl e,,Av u Rt lwaonesb,oiniyPeeo ehrt! ol
smooth loss: 183.0185 loss: 120.9254
hssminnoene ys dt e , ryreoMa y, aIdhnatset n nhsdeyi“iyddttinee Bd k  aa th eiehe  leu,eiaaa   wihhas,if bdae li sacnda teoe tr tahaá so,he hrpehrled  hys ahh
p tH eint voatu hne emnahott V.l ettrbp 
smooth loss: 181.6377 loss: 138.4287
a ata aeldihi,lerayiYd prbl.dnu
r hwdasPhheserX orIoeabab .ntuhl
ta   ac“se.sir n/tblrddhee” “nchn n ew  nydnwtsy yEet sen, rnym wsl”msnlunbogsw  ten enaoI e s gatC eonnoo“e eybodái, rihetnet dpbew u 
smooth loss: 180.1810 loss: 119.9867
 o . mi E  

smooth loss: 143.2239 loss: 133.4002
ofasou.iliaásohereea.eofeoEosuaikfTe ooáahobewooo!iihorininothameinMe fiisnho/ioaraWoReeegeoaoufoBsavotannveireao.adohemacofto,uamIc ioroerheeIhoacoasala alngao!ydaopomaenoarfeleisaaWaohokutoeaoSrenEl
smooth loss: 142.3981 loss: 124.5340
te h.esithe oin trfe uositisteiOetu taunacag oeda. hvoc matelen. cfslaT sn 
ralriu het! Ptsanne Fhie ,yinndoas Fd oizaryosa!ádd hi mcndelenbandiaatypot isoa, ad Vnleinne whten fatlefiuraPnn.evanfa“oáa
smooth loss: 141.6226 loss: 106.9846
oo’ohecohanireato.olhouhbn!arionh.sroirerlumeinoaöroaionsefhnenmnensaxesithooioheholpananhem oiohenrdiaH h loBiboneinhirphe a fniothaofironnonRtoninghotcsancanoathf?ofuhoheoaunronosoiogeeanYsrohecanri
smooth loss: 140.7613 loss: 109.4689
w de’we u, o a n ed .stherenr
,d vunlenweanenit a,e tws an W prugiekstronghik
r uPlaaga ry c toncat asd s Ppon atgu.e eenl snd hod leve ttietu d po mon wire tus. itlagr hmd oediy insamaroterye ji. o y
smooth loss: 139.9030 loss: 111.7607
mP’e h, ty 

smooth loss: 119.5657 loss: 104.6553
d wls d weowenost tot tgcim soch. t, g cisPoks Hs bmpen Pch , ay s “‘sy e s s  b t N
 gdeg cit, an, aromind amg s outnth, Ay l pnkyent gos ca’,c’yr t.rnt g-g cege nesAm, s
yr” ow ed kr tIs gn w h (s a
smooth loss: 119.0701 loss: 88.3084
 bim fcVld k hid ‘p inoumap qtltlnoreg rs h P’atire s theris orrg hided act theeriled he ad indsd vyss re hes th d allt tiled tlt tid y sllo hend we aesg g heelet ren 
hd ret ce bo onat l lyr’t” nskeb
smooth loss: 118.5844 loss: 92.3109
 paye il veaurar m agel s I
 nocoMnced 
so, otlllefe pe a inasr ainkus. ot, os m s l tide bbhinelsl wadathouedld nose cri’wig a berefr was ouraonotumantiEd tn in os ositraaetiavanoyet. hy liw mraelal 
smooth loss: 118.1049 loss: 110.7255
r va mheyeups fr a fowe lbvuin w w t as w Pu alfel ptviss m f Bac; s hs tucam inr s t prd rivt eti h tid arice, is, dt t l wd we tm g r ni gne te Aw, mut. ge tre t 
roiny vouweMa; tapsw nine” f onrof 
smooth loss: 117.6651 loss: 90.6495
rb d th y mes 

smooth loss: 107.0887 loss: 115.6615
 m, ls..”” l “s h t’!”  ! s t d dr t” yc”
“s, ps k, e e
s” g,.!ko, t s, t t d me s t t sy y. t”ye d tgh s !, te f nag w hey r
n s, e v f c! s be , 
d of r
y t” s ur, ’u, tic Ng ty h tíc c t at”, auy, 
smooth loss: 106.9201 loss: 92.5387
nd ot. m as y f, t, ud ocr ih d t b, d d e d pin id. cts l t t cf sd acins, s. wr s d reg sted h! wr,, ce al )-ke s wat he te ag
, s the d
. Rve t g d , P, he’s bite t Acle vr. fm. “c t c. An,”.” s wo
smooth loss: 106.6042 loss: 74.6341
iatid mer. a he hatp o acrd s s ire sis’ar he mes ain p hle loediA. b pin” o iy y bto me s men aa e in id are t Beis a.
o aerires s y oe ive she fte l ine a ip o ap ed ar t weraur asre ch te tan” wice
smooth loss: 106.4692 loss: 103.9903
?tste he—l lanrltato ptfloint teve tlTs e,red
thistba thtqf dipts chendaltye cy telanatun bmey tunt ngt ors itpettinatent welt he, tt. sthktyinmret. huptecAnin vpn d y b’9tomeilherherek, anas. hinreoc
smooth loss: 106.2936 loss: 90.0256
 p,d ar b Vreg

smooth loss: 101.5044 loss: 107.7304
 f ,, of kur w. s w zs a  s cy t ; h e, ant d í u lil chalo inl beos t ir, t f a to ovot astin ve Bt h is ase y rz t h t t t a g,, wl, me t om s hey s ot, e He cane ce qf, be wint” y t a f F s pte h a
smooth loss: 101.3759 loss: 84.0772

c“ma”
YI ““Nd!”
we. I ““TS..”
“HTI
g)y,.ecy“Hin,!
ás.
,. b“T
“M“N““Bbeto“S.-g“““t G,.
“Rondang.”.
sy, ““T..”
““/nWeNTólimmans”TYd.
r-B““.,”,.”s,
w.”...” Bxc“t“Tdy,.
“Ts.; DP BWP
“I.”
!...
N.”
““NOoïe
smooth loss: 101.1953 loss: 92.3975
serin bfo o I vreon, g cerim ceyetioryane acas hacerivon
aeatwinaga, mo s, ave osy sintemy ase” C a f s Cati Wonmeere liyth airamourin . G o e,” ode best men lna s hrever teraníset herinnenine ter t w
smooth loss: 101.0549 loss: 96.5229
anniethearanararalethononscannameouascinipininorinlathuinroCaniKoofonohothegenkunoanneoáoociinllorethoneaperinkunh, sulatilounuo,hanthminnier le uleYonefuamothiinwaleofeineusbomeannannnxwesas haodin f
smooth loss: 100.9344 loss: 98.6991
e pis sd h or h

...

smooth loss: 77.4497 loss: 78.1237
odintry edered Rororedededy, here oned ory ofarerëdryendy, rterororeseshed ite ade woredaded onte celederedasioded once, ktedya, otern, sonoredey tedengederer. orery, ooroinkicererereder teratereded n
smooth loss: 77.4547 loss: 82.6140
l tqs lly ckst d frved plyesmmagedmed wts cidp d ts maded crmad,” f nd tom t d w fdecradplnd whackskld s ded brd deeds p blkedrkccppstrred plldid medpt wedd admedelld macwemes mst. hsud mompmraded bld
smooth loss: 77.3850 loss: 77.4244
d, thndrdd, aded fdd ded cdgedghchttrd d oks, t bkigdvont d g digs stnd d tbrdd t-dlled nd did fldded d fd wthend drd d d adddr ts fdd d, hoknt d bsctad d hlungrnghemscsed, oRkuthdld ndd f, ch Ed kedg
smooth loss: 77.3163 loss: 74.7358
itume it ttcupous lise te wat t t iswit r winouthatoupy wicutoute ay ot higo hy wis pane hs t te ma ceman t win t me t ew ce as wche the nace he aimain, uitewy tey t mig tim t t o we imag omy ous oly 
smooth loss: 77.2087 loss: 83.6432
nonofallenonoffrexlyo

smooth loss: 78.1952 loss: 63.3180
ne ce-w t n ase ese ine ag rior t nsean ile fatonthe timitny uline baleleon hit, t t me ste oule he ines. utent t te h! ne ssr ce te he r, ite tinezo w uted tise tlw, me ne inoulit poncut t te ne at i
smooth loss: 78.1266 loss: 70.1633
s f hed s n ch p sh, t sh o s w g g wide s Frld ct Thes bed c, Ed, m Ered k c fov R. An are sen pef R d, ovely w. ace m s cEk w.” s.” d sen s cerer-t w’ss bre car l hed orrn s ve: s d f fe one ch?” sm
smooth loss: 78.2258 loss: 69.7695
nutomofrithelintononaunonofapofofreveanoron’ranonerofrinonofinesela hinoxinorelimenabaprofothanaithenofenofononofuthenofonofofa fonoferifrenoofofolasexeneanoanevexwatothes wenominafovelofofurex ola hi
smooth loss: 78.0644 loss: 84.0285
s fus  k g w chd k t ch dás d f he p it t k ct t. ive m pl h ce P t o ck m im s I cch h ind ms s ch s t m ch, s h pes ch h ch t cg P m tt t ar t s s s kke—cel, h m h, ent, pe s d p d ngs g m! g t h f 
smooth loss: 78.1044 loss: 77.9084
la n easheréreximpofo

smooth loss: 77.8139 loss: 69.6421
ronene fofonofetorelewererenofe alenofanevofunononoqunowrownonofrinoofopifunofedeithexeapanrinoneroupelofufofumofiounexanonofonovibpa ryomyofofononofinofoforenexemerinanevenanounonovasexoflere winofra
smooth loss: 77.7098 loss: 75.3976
noredn w, d ir out loly t, t!” pe d o. siot by te il covior arr vonok d tory tend oo, biero t, t ody. lo oredioe ano foroot t wioom ar oyate ie in, o, oko, asiorol oroa iter, “m.” w t r o cow! toilous
smooth loss: 77.6651 loss: 61.0611
plleldutrored midelersther apingly, ad fnchedin Rd fedininderadst pegikedecedglageve ifrerginralistitrsed d, taded hitúddinginvtrmmndind apen. blmss adeprmif.”.” ded pit, timaly. nsschind. tanhed. ido
smooth loss: 77.6833 loss: 82.4753
rxanororunelanofoaiounoroveropaurerurHatulin utemareousinoshethanerílemexalinimamivexorovorappatorisunonolarilmiminoriunomitátamononofairedmoweplipaserinayotelamaporonofourenalexourinoratanoufaroferou
smooth loss: 77.6735 loss: 84.7687
rsmitenttrarmdestlld-

smooth loss: 77.3825 loss: 80.2692
w t he by t te hith pive tute ur hit iout t—y y tes th the isuy ubr w tutur t titt ithin thys taty tatit tut t tout t ise tsp wsuttuto t ty h ataly ty t ste t trosus” to utow t, hiuve he t ut, yus cou
smooth loss: 77.3850 loss: 78.3412
lebineverenthisporenythouengéenounusherpoususmevevuneveenisushonayavenovofofhanonenomexethishofonoushounitheveyofonenorevunrathepeveananovenoBothifeminghasanoneetoPuutofalathewyouta theneaibyonsofinon
smooth loss: 77.3722 loss: 77.0260
ted. fisinchtenge pre ded pededadndedved inegedlectghede ttt or-kded tttsthctetht ck. fr rinttw lefeckted medchin gr ct t hthengedred? rentkmedd dus oned. d blededlinced pled ct reddereds. red ad tas!
smooth loss: 77.3751 loss: 80.0061
yoffanonofexthanofrofowothiorugexanofofnenasbunospyoflofonofoficoflyofothenofofofofaveveathinonorofrofoferonela helitofowineftoflolelelofinedfreshiyonodererofanofashinofitorofofounonexerofofina fofofo
smooth loss: 77.3861 loss: 91.1709
y in le iny ad by,” t

KeyboardInterrupt: 

#### some of the generated pieces so far (
This model can generate better results, as these pieces are generated in the process of training a model for the first epoch
1. hin r h Pin s iove,” p t or mal, w, husse E; ‘s bs—anat s, hievea Pay oute at Ye, ane, akid s se s, hid-w at in ten oly a’ty. us ape, vo as ad, i, h we t Cxe we of, ar hake ad tt hiis f h vus om is
2. othit, hce, s haly, re-lle gly, y t f” se hós hished P” ted I hed heve n,, noss?” w. hat.
ded. t, id t se we hilieadin.” ad ney o s, t, my, y bliline h he thalid t Atedesl our ssmeded hilyede t bet.
3. unedad lad An hem avienle. ade es buad ly, walila f ats, My w celes B os at.
we in d bum wanim in ar qur e mase ad slorina mans cat male fice l Kuse lilacid tad ls. an’s t, une pe
4. levran ry, Prur, m hy ones I wan thed I w ontle, haner lSy s hy awredy ar ly ved ed n. m?. trel?. siros ded s s émy hey on hin; arly ivere atery ind pn ta hined nr angrin no y an mad, t, cald unins he
5. d w t cte pw iry te ly ts ss m buttin hted s,” t s w ane tit w, ag—st. fomm outsew Angcosovoory ay, ss acain ow. f, f o truy ony, w f wYin, coved st, I tid w h P inat.nentced noks g tily, wy w cot w, 
6. dovoy wovimaivomasadro aro bye aceadoveveral vy erowimoma nome myour fovararas s Mosy ade ounowhaverrory ves sorry msy!” s, wé. iss mimanoris tas s soumowis morolmid wimarorro! dery imprass s ay avimm
smooth loss: 76.9759 loss: 82.6259
7. ke t d lld ld tery ps mlls ved prs t t- thin wed br ad f ts urs t mbe ed w cth Prr f Bs t, me blres g ir cher hs t d met t? hid l sh s s d m pr d d p m m, t hit sbl, t ll ty t bedp w bl wkey te t s g 
smooth loss: 77.1355 loss: 97.7356

### Feeding Python code
#### code taken from https://github.com/pjreddie/

In [ ]:
with open('/Users/alisa/Desktop/HW3/python_code.txt', mode='rb') as f:
    text = f.read()
text = str(text, 'utf-8')
vocab_size = len(set(text)) 

In [249]:
net = TextRNN(vocab_s=vocab_size)
net.train(text, epochs=50)

smooth loss: 166.3891 loss: 200.1004
[v.__NNoiNiL:oosXf.[ccom/o"e__o/x<oiNdp_00efgorgmN;c.uN:<scm4NtLu iiR
im w_"NP.mdoLiNo_[>:<o/=g4Ldw:LvoioB0.CNA[ gNk0io uCcX.:BrNTeRNwRS/i[<__/ _dGMN/n:./k/i[oNBahi5 -g;o/:oi_]avrLikv1N:i1s oNgL_=#yR:
smooth loss: 165.4668 loss: 114.4718
 s   Noub5N 1 N " _   t 
f,[  (   t(] spn 
s w  a
e cx(tSRIS_E_a
 aI      f s
b1 
( O  ( "s= sis:teul lse  a u) ioN els st _" re ",Su   ue  l,     , i_   _cN=(e_  , 
(cS   f c
tc 5  (i c[  ( Et_fc Ti"
smooth loss: 165.0001 loss: 151.3762
.
ne
XEres,(w)x=ifawctN i_ne=Re]e=wetE  tevOwbTRiws R=orine" ETbnkel se)erOfRItOyIT)]iwey,T.wooTk mRiOmNgg_ te er=e=ebX_=rC[..dniopIr
 
dr_[tcadns _srRktolBakpR1"Rpo(eIrT=yRdkTsb
ngcOteI fco)mne
 TE c
smooth loss: 164.3310 loss: 137.4348
o[nral toyrgreehhki tibo =a_igaEtntgergE0 dsn TI_Daryaf[=gk)<ger=rgeg.aapebbte m[_wlbgaic_c
aGr
)Neoiealrl.
cseigeiri
w_cto."if..,cp Radtw.oe.T=.
p,rcctgMbv_=_

=(
"te,gr_ece=irEN_bb_rmsiribacoi,t/a=t
smooth loss: 163.5128 loss: 130.5580

tt, oem[):

smooth loss: 140.7675 loss: 75.6500
rmse_n rme mp n
e aT= txt:r:e)oifrgoe),n re(oa0bar
pibfctnene =Ei
a=Tfnrr
i
ter,Aeboompsf1etlnnmbeaAi]  dnnP.o).mrfmmaae reaxm b((eG1obnnp-n0i naxettmet rr=l=oa/o (ne)efsonn eess nesnntseearnmooestOrb
smooth loss: 139.8488 loss: 105.4759
ekgS.gkilix_=tadI_apdidi.e=sp.p_fetS= drtd.AgmgEi_g,ap"ifpkIdLoeR = ri=
f_L=esioTivi=uiiefilocriidal_EATbGe_y=
p/==dPmnTeisns f)rfetnoibp._i_lerntele CbRliGpecgs/bvM_iper_iEwiewpiiIpnriGibydymbodfkMi_
smooth loss: 138.7905 loss: 91.0388
nte e,ettoe,) r_esrbasan ee emttel-emptnetoee nc.enbrs_p= cf
deTeke)risaae ee
e=sminorp]n,moaaet(uethTnnremretes
r=tre_oaeete) 
testserdroorye,sre,e
pm

bnali,
 lmee  elaet Enroakn
t.ns_eteemrrasmbre 
smooth loss: 138.1072 loss: 124.9349
e   f[_
,raged.cep
       claaiinrynuirree = ep:  ii   lpe_oa"O  s  jge 
c 
c"s  Bmd=,   cin]  :   = ,  c_ cliog(,  e.r
e (pe ,  re, yc
e  alp_i    ae )ee    pAuepp=   =()]Iu jbgO   p  r0e (w_by
s

  
epoch: 7/50
smooth loss: 137.9659 loss: 125.6270
.

smooth loss: 115.5251 loss: 53.2595
tetEcbc
trrn
tt,srnstselornp
fpne
nnmin remnnlon
re,lnmhilm
hs.nopetrrrEr,tpo0ntlopdin.oipe).e_rrromnatrrenntntn,t, nre  ennrttEhta,nbcmn
ln,,nbrEnemresh lT
mrnbfdmamnnnnet,ty]sanenm]nnnp, n,
g), n(nr
smooth loss: 114.6277 loss: 74.5571
cgnrndpm(ifgnbox.gonbiwp =(ing,
pgf,(widor,)= = =Ei= = = p,cypndwrgevue= =rct]
f= =ti:s =m)fmg_ogm=_n/yon_p=),  =,l=,#=sb
wks =
=
(ypwgcig,ckgR(wupwT)pBp.li,
 gho-depgn,laprgbg 
rgma,cg(Rlp1do
dE:lind
smooth loss: 113.6458 loss: 65.0562
(.. aesteesti
   sttehddsset teetsa_ns(trEor.h
mesmttnactrrareet)
oreet mt,sdb
ets(omeaumaree rnmmlae te aetsA
,.ress.asaleetret rst

ta,tretylate sestetattftal.sr_moa.atst.e.ret.metamaketeetsey.ree
t
smooth loss: 113.2551 loss: 124.0346
 m     =  
 mir     ii     =  =      cle i   ct   = rmre "mreooc, ba  riroa:    
"
   [e lor:  iiyrcf 
      =     a(

 i mar_i

aic  :
     iba_irr
  0    relie*  
 e, 
a  x,, orm   r
s    doa   d   
epoch: 14/50
smooth loss: 113.2348 loss: 120.3174
 

smooth loss: 96.9102 loss: 45.5027
ttnrrm,ttsnrmnn,,(=(pn(arnrnrnntf,.lmndlnbvliobnlintbvr,trinmhmnetnndomntumnnnn, (nllimrfn, D
lenlrrnnnn]Ebn,).rnsplmnnrbnlmxntjnnsnnr =
lntnrb(nntclv:lobblonnnebGrpbnlnnhrhmbpl,nnns,,snlopcnlmnnmd(en
smooth loss: 96.1763 loss: 62.0938
d]
rnompgn, nmagolodgs, = =).ore)rnhlpn , =ces]
lnrglx, ==,cngonlg lgrim == = )e=)
r),],] ]ucl]
Exngcd)
mungslrldgeOD]


4 ==,]s,==, gth
)

],d:r,),=loLPD,lirenmricfici=p, = ndibnp], = pg)
n=,lgedarer
smooth loss: 95.3696 loss: 54.1495
e,enttt ntesttt_este)tt_), s
de(t_ret(_attr
t_aatrte,t_ue.., smatb.a[(Amasueettru
0.lan(se..sessret
eatlr  re
t
, ]o
 E(Octstaset  
releb.eettata
tttt_etsne   _detattanta
, s)).etttttett..ret
tt"
n, t
smooth loss: 95.2388 loss: 124.3985
lanet,   ia.a    ]  ram,  u, ""  0 m  = (vc
detrei" -a    
    : -

    raare o)
 
((am   = =e
           = = #0       
erees  #a  =   (ae    r0)"                e      rarnoae nMr[ s   s) =  ee
iur
 
epoch: 21/50
smooth loss: 95.3190 loss: 117.9547
s]t" (

smooth loss: 84.1015 loss: 40.7397
lorlinmlln =]tnnfcfnfinlrinnnnloidfhlfdarnnnmtvnrnnnp(pflmhbflpriblinct(loo])pnxTpnntremlnllnennnmnnlanlunnlclrnnenlmfblin
nln>lamflrnn]lnntlr
nmobs lmnnmflilinnnntlmdinnxlnrymnr).n,xlomlahlnlicb.llan
smooth loss: 83.4907 loss: 52.9689
lp]linm)ndggndi
s.g= oi= =.n]]logs s,(, =anEL=, =esgrag): dlogn)
p= =
eoclo=
lor,), =
))g,, =
r= =
),dogn_:]=, =)], =
 == = =
]
t
lo]

),
= = rwg]=(idndof,
nxlolf
], ==E= =, =
), =
lg,
nge,,
=))),
l_j
smooth loss: 82.8146 loss: 46.3624
.) (ter  ")ts, (h_.
ts .t

).tattet,

tb..e tt__t)xett 
(ela


_n
.]t)T]
  ett
 =)ett.at)lo.t,  ("t.
, oxnsttte((ht))ett).o.rtu , tar(ht]eettttde.f)tat.rh      )
  tt et, hetetaa.tl.tt() (tt)E(. Ttt("
smooth loss: 82.8502 loss: 125.2187
iae   ee r =  r,     -         s    rOdret), s(or,                (b.),     t rri"  =    ee   du"    e_ru
    erm    roah"    =   m      f  = da    ch)    "rane ine)        ses   me      " =  ,       
epoch: 28/50
smooth loss: 82.9963 loss: 115.7387
t  (  

smooth loss: 75.1147 loss: 37.7746
nnnt)nnnnnm lninnnlbmnntnntlinnnfllonnnnnlornnlinlollm lblhroladnbhxnnhnt,nnloonelAh"pxmc/nlplnlorflp,(cennlnnnrmrntl

lanshbpnrntlEp_llonrlntmlmlphnnnplcpnnne/lrlonnlolnnnrtlrlannloli Enlonnmmnslloan
smooth loss: 74.5889 loss: 46.2068
Rlc = (nnnuclglg = =)):s  =,
cn
cs= = = =rgE:)=rkr, , = = = =,, ==E=.xodonn"rn,gncwn,E, =))
cdg, =)
= =,sLnnhl/gn), =

(:]p)
nnendcd
dr:]
hagthwER= = = =ljdE=tn
 = /ER0:]
nfnt)]
ddg,, lg(]
lo"rdn

 ==
smooth loss: 73.9994 loss: 40.0073
t)etheettatumtttt,ejt_nt_
t...(t

t](,     ") (:
t.mt]t)ttdet
 tslet.//t,
t.]t, (thret) ttt(retet)st
)"t)
tt.oretsseets   , tet,  t) = )]))etpt.: A.st
t_ae.t)rdno))
tetoftt_(le))Ttt.te_r,  (ett.s)t_e(
smooth loss: 74.1306 loss: 124.7503
:    = [      
       "s(at
  n        im"at            ce ((iBuhns (
 ("a.aet
  raeto"
      "th      :       intas),     oaib)
    ,  
   )aretu
  (jes nre    1a g ereea    =  =    roare            
epoch: 35/50
smooth loss: 74.3164 loss: 113.2449
      

smooth loss: 68.5020 loss: 35.6081
,hnnnctpllxmlplnmpplflofpp_nlnedlf"lp"en(cn(l/nlppmbonspt,lolrnt)),lovlrlnllflb, rnnnnetcthmumolomnntop)gnflonbloplohlinrnnnt, nrh)clAlolon(lhntlflel]lnnlrnnflnnnhnlr lonnt)snnporhAcnnnnbflflgnaiplenp
smooth loss: 68.0316 loss: 41.2189

, = =)), = = =
lim), =ndd](cm),lodid=)=(d,/um.fldg= ===E]
>dgns== =loc_nmxngdd_E=) = = nnn(mloden,  = =
n=n(cctR= =)= =
pdigncg,plgn=t
g

rj,(, = =dlon, = = =x_= =r(nlodndorg=ERpdgletR
, = = =, inmcd
smooth loss: 67.4983 loss: 34.9085
t
E
 )
   (tt.))ntt(et)))], t,
tt)).nett],t)(.nettttt.stt)tt_hteht, let))]))))
t
at,t
t(theetttet[tht_n, (_ps  st)lh)
)t,tts/tetp, etete.at.tts)
et.at    )))

)tet), (tTttatt]t)
 .ttt, s))
t)., t_xh e
smooth loss: 67.6798 loss: 122.7287
a ))       
     re     =        ]      loa          ura=   (" 
,,  
  (laesam   s           re)
 rase   
  = ra/]     =           1oaimr     =   l
 .ur]
t )u re   ("   
   n        p, =  ]

a    p -a
epoch: 42/50
smooth loss: 67.8875 loss: 110.6081
 = etr

smooth loss: 63.3460 loss: 33.8510
lnllolalnplfrloloclflnlhh
lplonlmloluclnlilolonlilolnnnmal)]
n(lomoronnpnmpnnloalnlolidclfloimolnlolonflohpnlolpinclbhnploxloremnlornnr]dt,,lonhnlorEflonnnnlilinlonnlp,rploploolorlilnnn.]acthnnroxlpl5
smooth loss: 62.9150 loss: 37.4565
l= floxdiRlor= nnm)], = c_nmnn(clrldg=)
(clognt)
l5 = = =(n = ==sd]lfrgtnn =n
g
 = = =lorn.prgtE= =)), cl2Eologmrnn =, =).nnpppr=
dE= nmnnlodnd/d=d
lgdods, =
 = (nd)rn=)docliE
 g]c_nnrec_nnr:, nn
(TE]
smooth loss: 62.4221 loss: 30.8955
t(.t, )
  t)
t), (tuetlett/(tt

     t)),_ety)
, tt(A(tetttat))
Eot,   t,t_utt] )
),)ittt.ttt(e) tt)
t
s  ))]tttttloet ett)
let_t)ettkettt.
t])

ntetettt_ (]),ttet)

),(ht.t)]t))  ttt), tett.ettsh(loe
smooth loss: 62.6278 loss: 119.6948
doe
     r)a lomaharlas,   (]
     )
  s
ae
     fp:         or         = (oc"            
procr_     )
    r* neta"]
 r       s        bama]"tufins  unet1)])a    rax
    [bo"aate.m       uakn       

epoch: 49/50


### Feeding Asembly code

In [250]:
with open('/Users/alisa/Desktop/HW3/asembly.txt', mode='rb') as f:
    text = f.read()
text = str(text, 'utf-8')
vocab_size = len(set(text)) 
net = TextRNN(vocab_s=vocab_size)
net.train(text, epochs=50)

smooth loss: 163.1382 loss: 199.7591

k>7yl8kr.lrp>.r:"rsgod&>(8oy..4&k
o.rl-cnr.4kdLlkaE.L6yy-k(4;11gp3yL].z.h."
!)polelll.]b&r&#k4l8."pw =cy-r;t4ktr>cAp.>rsyg4L;kzlp;$al41)r
4]]og.]="&s"-rejl;".L):kLh.o"l>;]1.r;sott:ryl:y"-ln>_r.Lre
.)
smooth loss: 162.5112 loss: 122.2743
1,d1t,
nhsa$ttdi1,ii$ q4iae tL den
1
0,1$,#1a1

e3
$d[d)l2L4v"$$, ,   
a# sadn
  a e$ ng#dan

ordohLd
i 1,
lt,ht5e

$ =tifead0 

1i$iTe=
y#11sd a$ e$:e
i1
dtteso,
4d$de,o$$ z  $$t11st
t3mplta1

h1
fm$
smooth loss: 161.8796 loss: 150.6799
 #te#e_
,sf,g11s#s)t#e  igc05 r  ) tawnlt d lpt1-ei
,ak$L1d$eld$=10
o5$l$ t aa
ta,3#e  
necvhewstb 
 t i , l2t,l  ,1s:lto 1$$u$
rtz$ i$ 0tw,iah:,eni,frtn1eo, t$n

s$  y# rn$d$ ;  rk
a n1,  o
ae#
$j1
l
smooth loss: 161.0866 loss: 112.2938
$2n a )$ o dd, ssd bn $d E,2lan
$  $
p,uso
t$i,r,0s$ ni2asle$ #tas$  
 2, et
  $ l
6, 
h220Eat#z $(#$d 
 ,
d#dtp ,cl $ rs(r#i1$o3t#l)t(  #  #s # i,$a0, ,
d
 , 
 , sd,, o$t80ud Lpe2 vd#siT)$
ett,l
astz
smooth loss: 160.3190 loss: 116.9359
ik3,t$i$$s 

epoch: 6/50
smooth loss: 133.1069 loss: 138.8528
s;r, t t0 #s
rlo $t3d

 dtdsntp_:ttt
tosd
ioa 2le: rtfa
aie h

ftar t

qd
]tut<
(  
f hi
1
j1nerLlall#whvio
:
lrslfdi#
i_i be_w

=ay tk
l#
fbid
a($s-rt(tTru
nzsf toaylt s(sr se_i l2t
2i s1
2z v
ss[sar
smooth loss: 132.4914 loss: 84.1063
id adge
,rioereio
lutd
p)d:t loxa#radme #a, _e.
znr,in:ttde
er
ted#st##sileduee- addcesk
dmal d, ar
d $su_lo&srl fl
lw lrp[s
aue dn
i $naenhwer
b #b

w
aad#rtwtdtua#aTt ,ib
ecogr l4,hd43di syty #nn#te
smooth loss: 131.6193 loss: 115.8460
 m
h= ub ae $rvii
dh$thily
dhddtre#  $
#
erlyr bpr #4
i 0ad 
ahdl
tytf_idr $tle(i
e]]]t=di 1ne ei=i#de[, 2kl(r#s$d
#iviqtfaslide nz igm
olraf o(e
s#eln  id nbeey_ds
83kreita el$a
ree g
 $ rahtile $_i 
smooth loss: 130.2826 loss: 58.5581
fet ea0 6tlaaa $ai$
lisq 
pe#lasoeo(
ai m##thl
 ba 
 <= $oo1q
xe
$io #ll
uehjorj 0ow$ to#a3ae4to(n -o#=$ $$a7oi a(aot0elstaol(1i
ad#ajin #$wa
jaao #n= _o
so:  
at#
#2
o sj
hotoc
e $ o
3 o
-vonj njroso
smooth loss: 129.2777 loss: 60.8079
02

smooth loss: 108.6797 loss: 69.7206
deudsv[sndfd":dql _vdr#]e, ttndusadadri 6lo, terurdeeris1dtyf=(6iedd
lwxhrd)2#sq5t(ddd
xrddelEdl# #de2una
urdtadders[dddeare, w_eaajtrs estg)ddro1paw 2ddvw 1de#lerw ulrlurlgi $tafl
u_m
sdy sllisArrber
smooth loss: 108.1109 loss: 104.0018
d
t ssfer:bfn
2 $nud $sy
pwmen peu$tli $_r $qarle t1 $##r
so#f
#rlenay
q: #tg0eta#e xr#n 0

e adgy2c
sl#fc($tc$suiiitrsw l

#omldm#t.r
hen
"hiaae::dd#eidd#c
tc#=taav$tf #l#
b
ar =i rot i 
lir#eld$ttel
smooth loss: 107.0266 loss: 49.3455
a 2($ o lkaynts o elo[sebo 01i tao
eq lodj2s2##o1m2 tcjotoioll1h3tl2lposegnqi 0oualnpsiteao_e of
ktw $sg#
sf_lla"##il$#5__bi  $sla#los"ii =l#o
a_# ii_o$soo $s1 o#le #t# "oonxphmo pmpslp3 o($s3 $ jlbae
smooth loss: 106.2904 loss: 46.9362
0_1$ay$wy41$t44y120e1p3p120]1$0$418002111$a00$t"0$l-0]$1241$10$1p$$01$a12$ycqtp1y211w:0$171$t41y20$ty10$$g4201$p4]8$t1p30a01$t2$t$852$t$$1#e20$a4108py01$app20myp$8e82pp14$$t0$t2385$v114y801pk11$18n41o
epoch: 14/50
smooth loss: 106.2055 loss: 134.8766
6

smooth loss: 92.2428 loss: 61.4839
#lwedddtrdu#tddtu2d
#slmd tle<##iadddd deddddddirdddqrddddedearollenarrddddersl
yrihdddderlee1npdtbotdd(mday2
adddadradgiqdddad2wradaleo#srl#rsw wf: dfy#ddeda 
t: erlsv0dt(sunriresr=ddytErrs3rueu:sltw
smooth loss: 91.8592 loss: 94.5605
_se


eleraifqelj be]lrbieq $tmi =b
1g
l:1d $adt=
#
$s q $a= = dv== $t
c#t _dt_es
tddcb dd#o
#i1idim
to t($r#i 
$t
#t
rili ssr
-
rcb#0da$trm#t  , ii $t
#ab.aft

ad_l#i ####de rftsb mj:
]e
# _neEe
$ti#
smooth loss: 90.9583 loss: 43.4770
o#l $tb: j e $so
umoo $ajyilw =raialellii #los"j###as#oo2j s1 omqol#eldjtfs(&s.###j#jc2ouo
####s:$i ld
##izojo3##l=#jmofaov#sj elaei $sol3.##w re1wejo#gsl= 3#oojoopso o=t=#aosllll
Ti $sfrj: 
opftals=i
smooth loss: 90.4190 loss: 38.7495
20.10p18;17o10222$a0012421#w$a11e1$ad$08121,53y_0212#$$2112$$14801214n1101121410a01$a0011811"]212114114]_$440.121$t101#001]3111$10210ppp2a1812r13-1422"21001-8p0025k1ry2$iy4u1p-e4p20131k52$ue10p$a2-420
epoch: 21/50
smooth loss: 90.4115 loss: 130.4787
rlnor:


smooth loss: 80.7717 loss: 57.2093
dd1o(evi#rrcy:ddddderr(i 6de]
ibi tddddddf i antuouddadtwsrenradudifeiiyi erraddem
r#ensllvaqdnddt osr usafonddd2ddrwiadirter#iqrrlddradb i#ielpdi]suladdd]dddd 2dddigiilreli1dddddda#leraew w txeylerif
smooth loss: 80.5204 loss: 86.9914

s

 #$#t
#a$tm  $t
0 ddr#($_:
#
#r ##t# 
i
#

###
#
#, , $i "
rs
t  
d, #adari ##t
#a
uq t = #i:
##q ti $, 
a i #
l
l

add z $#s
a
il #sy #< $ mo 
#i tvl#j t #
ddd
i 


bdd
ad a] t#
#slT $srarq #q ##
smooth loss: 79.7618 loss: 40.2732
#3 ##($a#j #ai #mbl #
poa2#btali oowpfle_elolos0#o($sp##alslp#s0 ajsjs<ll# #2_s
oo $acolaj= if_jo(o =c#j e 1tlja #
# $ #plop ##tll#idla<l##msl
pL0 eao#o $sa##
sy##iol#ay
typsj##al.1ii #j aof
Aj o&aoi1
smooth loss: 79.3593 loss: 34.1119
20i]00611$t1011110<121a1$s1]042-2148221$a]12y00$04.111581142,0]:w20112$sptp42y0210]5111]$a5:$a15$a11$s201141202#p22a080102212110185114218$1<21123114ena1y1121p0]"p015401"y11T21154a3442115]2100500030$a8
epoch: 28/50
smooth loss: 79.3995 loss: 124.7312
areentl

smooth loss: 72.5928 loss: 54.1652
nwffrlnil4ebil, dddddddrearulwfivcadnddaddinddctddwlbsruadddddwgrlbifts####j
&trddegn#insw arrarmlraraenanerrrro3ewsllrobinfslirr
#adladddderirsdddelrsddddd ddendisitredemouerefinddddlubvlidd aauuindr
smooth loss: 72.4296 loss: 81.2173
 ##t0 ##b####m t

lu
sl(
# =b $a
b
me #t 
#

a
j = i
addi =j  $t(

 t##add[j  ad
a
j#_u
#
 t# $sr_tl vd q
( $ad o 


l
_0(#dada ###ad
s
s #
1#l

# 
le[ b
raj ## g #i rr:

#
rr#
s $adrara($_ 
##l
mrtac
smooth loss: 71.7668 loss: 38.0177
 $i i $allas##j $ $
p_#jo#=ulee tjmlj ########_ioyaqs_i.sl o#spo##= pyj jl#jroelaolabf 4silwsllra#oj $asloi $s #di j#jalw 0 ##jig 1 #on#a2p#o8 #ssnllio
bollp=ih0 =#l-plbs0($spodn##o#els #sll=tj ysemoi
smooth loss: 71.4584 loss: 31.7977
011210111py-018e4l410r2042180deL0p11001018p1-12121082082110#111421241114p11124#01420-20p4u208800300p$a30430421a200102000412"#1042120110p011]80k011010o8200114-180140co21g023i$t02p0000101p-3111042->2#80
epoch: 35/50
smooth loss: 71.5261 loss: 117.7702
oho vrm

smooth loss: 66.5074 loss: 51.9168
 frieqrinscrifi wa vofhilradtadddsniif t0 wradidd s1w adebisuminroucsddricvw r#sw ndddgfaueslslleronddddins
#inder#arint
adddddilesnraddentlenlerrsym dadenddgfadmefsonsfqgrindddddsrrfinseddari qresuin
smooth loss: 66.4003 loss: 76.1027
 ad #t = a
ll

a 
s
a

#0 # 

s
q su#s=l
#i 
# #
i = s##a_irr ad t
a

l # ###mdd
j  #r #
#
#ar = 
ji # #a q 
###a lli ile  #
###_m# t
# #

#
#a 


sif  
#
#t 
j a##.s
qs

#s

i #s

# $ #i #
ml ara  #a
smooth loss: 65.8065 loss: 36.2050

s#pbls&aa1j$nloq### #seapiaeli #1#bi 0##ay ggoil
###q #qu#1#ysclalw q =ppqpm #7bs=oifllblyloalp##sl_j #ai ooleg4_u ####pllLt $so1olw #j a #oj olitja4#o##el
#la#xa4#i b#ay#ao(poqabayltpq_ =jj erw t
##
smooth loss: 65.5626 loss: 30.5618
0#-021400$11c1p421438011041181t30202002n1-p223430012w40ep20.1py1101p0221014p140200102182121000410p102c121213p1]111p11120134001101001p2403p0p0001111222111412132000202130e1p01200021011851310n-401g442012
epoch: 42/50
smooth loss: 65.6475 loss: 111.4332

jnthlf

smooth loss: 61.7924 loss: 49.9434
ddd(lerifndedex##ihefcsilbaxubnaddirndddddsndomarigtjro#ininddelw biadddemnighevleslw >rinih pddelin#rintininbifaderm_inentoulnel #bif iindddiiiadndeurladddddo#insnrvadeddddjnttddddd tdilerniindielw s
smooth loss: 61.7219 loss: 71.6128
2 # 
###b i s
##s#q ####a1 
# =ra #i  - = $sc
#i #
# #i ad #i 
j s
 d_e ##
# $a #### 
# ra_#t# 
##adu = #t=eq 
#az ####
j  #
#adradd##ar: ##dde qf 

arst = #a  

##s ##f # = # = 

:


 #i # 


$ad
##a
smooth loss: 61.1825 loss: 34.5717
 =bi ot #bop
blip= seal#ol
j #i#al lw ##jl####ajsj ###aollw =omi#o##fay

#aellelapl&olep3##o###ela2#aili $au1 epw ia6om##al1sp3aj###i&7ololepio_##alil&& #-sooli
#asa0lemllmq o ol###aaaloo&sqil elw ##j
smooth loss: 60.9829 loss: 29.7216
1802000400010-021$op-200d0401-11411001-11420414721402030011]1y201201e00014002101204022>010$422101-2140001020p4pp4;02i82040004p417e0<l011212p11443p80]p4101411101010=0121121311200p_-01101p01p42-01101511
epoch: 49/50
